# 05 - Streaming API

**Por keywords**: crear un stream con reglas y escuchar resultados (solo blogs/foros/noticias). **Por proyecto**: escuchar menciones de un proyecto en tiempo real (incluye redes sociales). Con token **demo** puedes probar streams por keywords (queries: cats, dogs, cats AND dogs).

## 1. Configuración

In [ ]:
# @title Credenciales { display-mode: "form" }
ACCESS_TOKEN = "demo"  # @param {type:"string"}
PROJECT_ID = ""         # @param {type:"string"} (solo para escuchar proyecto)

BASE_URL = "https://api.talkwalker.com"
import requests
import json

def tw_put(endpoint, data, params=None):
    p = {"access_token": ACCESS_TOKEN}
    if params:
        p.update(params)
    r = requests.put(f"{BASE_URL}{endpoint}", params=p, json=data)
    r.raise_for_status()
    return r.json()

def tw_get_stream(endpoint, params=None, max_hits=3):
    p = {"access_token": ACCESS_TOKEN, "max_hits": max_hits}
    if params:
        p.update(params)
    r = requests.get(f"{BASE_URL}{endpoint}", params=p, stream=True, timeout=60)
    r.raise_for_status()
    return r

print("✅ Listo.")

## 2. Crear stream por keywords (solo con token propio; demo no permite PUT stream)

In [ ]:
stream_id = "workshop-stream-1"
payload = {
    "rules": [
        {"rule_id": "r1", "query": "cats"},
        {"rule_id": "r2", "query": "dogs"}
    ]
}
try:
    if ACCESS_TOKEN != "demo":
        tw_put(f"/api/v3/stream/s/{stream_id}", payload)
        print("✅ Stream creado/actualizado:", stream_id)
    else:
        print("Con token demo no se puede crear stream; usa tu token para crear y luego escuchar.")
except Exception as e:
    print("Error:", e)

## 3. Escuchar stream (con max_hits para no quedarse colgado)

Con **demo** puedes escuchar un stream predefinido; la API devuelve chunks. Aquí limitamos a 2 resultados.

In [ ]:
stream_id_listen = "workshop-stream-1" if ACCESS_TOKEN != "demo" else "stream_1"
try:
    r = tw_get_stream(f"/api/v3/stream/s/{stream_id_listen}/results", max_hits=2)
    count = 0
    for line in r.iter_lines():
        if line:
            chunk = json.loads(line)
            if chunk.get("chunk_type") == "CT_RESULT":
                count += 1
                data = chunk.get("chunk_result", {}).get("data", {}).get("data", {})
                print(f"Resultado {count}:", data.get("title", "")[:50])
            if count >= 2:
                break
    if count == 0:
        print("Sin resultados en este intervalo (o stream no existe).")
except Exception as e:
    print("Error (¿stream existe?):", e)

## 4. Escuchar proyecto (solo con PROJECT_ID y token propio)

In [ ]:
if PROJECT_ID and ACCESS_TOKEN != "demo":
    try:
        r = tw_get_stream(f"/api/v3/stream/p/{PROJECT_ID}/results", max_hits=1)
        for line in r.iter_lines():
            if line:
                chunk = json.loads(line)
                if chunk.get("chunk_type") == "CT_RESULT":
                    data = chunk.get("chunk_result", {}).get("data", {}).get("data", {})
                    print("Muestra:", data.get("title", "")[:60])
                    break
    except Exception as e:
        print("Error:", e)
else:
    print("Configura PROJECT_ID y token propio para escuchar proyecto.")